## Speeding up Transformer training:

Strategies implemented:

1. AMP

2. Dynamic padding (using `pad_sequence` and no manual pad count calculated)

3. One Cycle Policy

4. Parameter sharing

5. Scaled Dot Product Attention (SDP Kernel context)

6. Gradient Accumulation (not used here)

7. Converted encoder_input and decoder_input Tensors in `_ _ getitem_ _` from `int64` to `int32`

8. optimizer.zero_grad(set_to_none=True)

9. .to(device, non_blocking=True) (where applicable)

10. Was able to use `batch_size = 64`

11. Reducing the hidden layer in feed forward network 128

12. Removing english sentences with > 150 tokens and french sentences with tokens > 150 + 10


In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/

/content/drive/My Drive


In [3]:
rm -rf ERA1

In [4]:
!git clone https://github.com/MANU-CHAUHAN/ERA1

Cloning into 'ERA1'...
remote: Enumerating objects: 519, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 519 (delta 22), reused 48 (delta 16), pack-reused 462
Receiving objects: 100% (519/519), 14.23 MiB | 14.98 MiB/s, done.
Resolving deltas: 100% (222/222), done.


In [5]:
cd ERA1

/content/drive/MyDrive/ERA1


In [6]:
!ls

experiments  main.py	README.md	  resources  s12  s16  S9	utils.py
logs	     models.py	requirements.txt  S10	     S15  S7   test.py	webapp.py


In [7]:
cd s16/transformer_pytorch

/content/drive/MyDrive/ERA1/s16/transformer_pytorch


In [8]:
!pip install -q -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.4 MB/s eta 0:00:00


In [9]:
ls

config.py   model.py          run.ipynb  test_run.py        tokenizer_fr.json
dataset.py  requirements.txt  runs/      tokenizer_en.json  train.py


In [10]:
from config import get_config
import torch

cfg = get_config()

cfg

{'batch_size': 32,
 'num_epochs': 10,
 'lr': 0.001,
 'max_lr': 0.01,
 'pct_start': 0.1,
 'initial_div_factor': 10,
 'final_div_factor': 10,
 'anneal_strategy': 'linear',
 'three_phase': True,
 'seq_len': 500,
 'd_model': 512,
 'lang_src': 'en',
 'lang_tgt': 'fr',
 'model_folder': 'weights',
 'model_basename': 'tmodel_',
 'preload': False,
 'tokenizer_file': 'tokenizer_{0}.json',
 'experiment_name': 'runs/tmodel',
 'enable_amp': True,
 'd_ff': 512,
 'N': 6,
 'h': 8,
 'param_sharing': True,
 'gradient_accumulation': False,
 'accumulation_steps': 4}

In [11]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
cfg['batch_size'] = 64
cfg['preload'] = False
cfg['num_epochs'] = 90
cfg['d_model'] = 512
cfg['d_ff'] = 128
cfg['pct_start'] = 0.1
cfg['max_lr'] = 15**-3
cfg['initial_div_factor'] = 20
cfg['final_div_factor'] = 30
# cfg['gradient_accumulation'] = True
# cfg['gradient_accumulation_steps'] = 40

from train import train_model

train_model(config=cfg)

👀Using device: cuda


Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

Max len of source sentence: 471
Max len of target sentence: 482

⚡️⚡️Number of model parameters: 56,337,490



Processing Epoch: 00:   0%|          | 0/1788 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Processing Epoch: 24:  93%|█████████▎| 1662/1788 [08:51<00:40,  3.12it/s, loss=1.55427, lr=[1.3436190176987307e-05]]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-2cb2c385aa34>", line 15, in <cell line: 15>
    train_model(config=cfg)
  File "/content/drive/MyDrive/ERA1/s16/transformer_pytorch/train.py", line 321, in train_model
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/writer.py", line 1233, in flush
    writer.flush()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/writer.py", line 146, in flush
    self.event_writer.flush()
  File "/usr/local/lib/python3.10/dist-packages/tensorboard/summary/writer/event_file_writer.py", line 125, in flush
    self._async_writer.flush()
  File "/usr/local/lib/python3.10/dist-packages/tensorboard/summary/writer/event_file_writer.py", line 190, in flush
    self._writer.flush()
  File "/usr/local/lib/python3.10/dist-packages/tensorboard/summa

In [ ]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
# with torch.backends.cuda.sdp_kernel(enable_flash=True, enable_math=False, enable_mem_efficient=True):
#   print("pass")


pass
